In [1]:
import tkinter as tk
from tkinter import messagebox
import math
import heapq
from collections import Counter

In [2]:
def calculate_probability(text):
    probabilities = {}
    total_chars = len(text)
    for char in text:
        if char in probabilities:
            probabilities[char] += 1
        else:
            probabilities[char] = 1
    for char, count in probabilities.items():
        probabilities[char] = count / total_chars
    return probabilities

# Huffman

In [3]:
class HuffmanNode:
    def __init__(self, char, frequency):
        self.char = char
        self.frequency = frequency
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.frequency < other.frequency


def calculate_entropy(probabilities):
    entropy = 0
    for prob in probabilities.values():
        entropy += prob * math.log2(1 / prob)
    return entropy

def build_huffman_tree(probabilities):
    priority_queue = [HuffmanNode(char, freq) for char, freq in probabilities.items()]
    heapq.heapify(priority_queue)
    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        merged = HuffmanNode(None, left.frequency + right.frequency)
        merged.left = left
        merged.right = right
        heapq.heappush(priority_queue, merged)
    return priority_queue[0]

def generate_huffman_codes(root, current_code, codes):
    if root is None:
        return
    if root.char is not None:
        codes[root.char] = current_code
        return
    generate_huffman_codes(root.left, current_code + "0", codes)
    generate_huffman_codes(root.right, current_code + "1", codes)

def encode_huffman(text, codes):
    encoded_text = ""
    for char in text:
        if char in codes:
            encoded_text += codes[char]
        else:
            # If the character is not in the Huffman codes, raise an error or handle it accordingly
            raise ValueError(f"Character '{char}' cannot be encoded using Huffman coding.")
    return encoded_text

def decode_huffman(encoded_text, huffman_tree):
    decoded_text = ""
    current_node = huffman_tree
    for bit in encoded_text:
        if bit == "0":
            current_node = current_node.left
        else:
            current_node = current_node.right
        if current_node.char is not None:
            decoded_text += current_node.char
            current_node = huffman_tree
    return decoded_text

def calculate_average_length(probabilities, codes):
    average_length = sum(prob * len(codes[char]) for char, prob in probabilities.items())
    return average_length

def calculate_efficiency(average_length, entropy):
    return (entropy / average_length) * 100

def process_huffman_text(operation):
    text = get_input_text()
    if not text:
        messagebox.showwarning("Warning", "Please enter some text.")
        return

    probabilities = calculate_probability(text)
    
     # Check if there are characters in the text that cannot be compared using Huffman coding
    invalid_chars = [char for char in text if char not in probabilities]
    if invalid_chars:
        messagebox.showwarning("Warning", f"The following characters cannot be compared using Huffman coding: {', '.join(invalid_chars)}")
        return
    
    huffman_tree = build_huffman_tree(probabilities)
    huffman_codes = {}
    generate_huffman_codes(huffman_tree, "", huffman_codes)
    encoded_text = encode_huffman(text, huffman_codes)

    if operation == "compress":
        decoded_text = decode_huffman(encoded_text, huffman_tree)
        additional_info = f"Decoded text: {decoded_text}\n"
    else:
        additional_info = ""

    bits_before, bits_after = len(text) * 8, len(encoded_text)
    reduction_in_bits = bits_before - bits_after
    compression_ratio = bits_before / bits_after
    compression_ratio_percentage = (1 - (bits_after / bits_before)) * 100 if bits_before != 0 else 0 
    space_usage_ratio = (bits_after / bits_before) * 100
    entropy = calculate_entropy(probabilities)
    average_length = calculate_average_length(probabilities, huffman_codes)
    efficiency = calculate_efficiency(average_length, entropy)
    efficiency_message = "Compression was effective." if bits_after < bits_before else "Compression was not effective."
    
    output_text = f"Original text: {text}\n"
    output_text += f"Encoded text: {encoded_text}\n"
    output_text += additional_info
    output_text += f"Bits before encoding: {bits_before}\n"
    output_text += f"Bits after encoding: {bits_after}\n"
    output_text += f"Reduction in bits: {reduction_in_bits}\n"
    output_text += f"Compression ratio : {compression_ratio}\n"
    output_text += f"Compression ratio percentage(%): {compression_ratio_percentage:.2f}%\n"
    output_text += f"Space usage ratio (%): {space_usage_ratio:.2f}%\n"
    output_text += "Probability of occurrence for each character:\n"
    for char, prob in probabilities.items():
        output_text += f"- {char}: {prob}\n"
    output_text += f"Entropy: {entropy}\n"
    output_text += f"Average length: {average_length}\n"
    output_text += f"Efficiency of this message: {efficiency}\n"
    output_text += f"Efficiency message: {efficiency_message}\n"

    update_output_text(output_text)

def get_input_text():
    return text_input.get("1.0", tk.END).strip()

def update_output_text(text):
    text_output.delete("1.0", tk.END)
    text_output.insert("1.0", text)

# RLE

In [4]:
def encode_rle(text):
    encoded_text = ''
    count = 1
    prev_char = ''
    no_of_vec = 0
    max_count = 0
    if not text:
        return '', 0, 0, {}, 0, 0, 0

    for char in text:
        if char != prev_char:
            no_of_vec += 1
            if prev_char:
                encoded_text += str(count) + prev_char
            count = 1
            prev_char = char
        else:
            count += 1
        if max_count < count:
            max_count = count
    encoded_text += str(count) + prev_char
    
    bits_before = len(text) * 8
    bits_after = no_of_vec * ( 8 + (math.ceil(math.log(max_count+1, 2))) )
    compression_ratio = (bits_before / bits_after)*100
    probabilities = calculate_probability(text)
    entropy = calculate_entropy(probabilities)
    average_length = calculate_average_length_rle(probabilities, encoded_text)
    efficiency = calculate_efficiency(average_length, entropy)

    return encoded_text, bits_before, bits_after, compression_ratio, probabilities, entropy, average_length, efficiency

def calculate_average_length_rle(probabilities, encoded_text):
    # In RLE, each character is followed by its count, so the average length
    # is the sum of the lengths of the encoded characters.
    average_length = len(encoded_text)
    return average_length

def decode_rle(encoded_text):
    decoded_text = ''
    count = ''
    for char in encoded_text:
        if char.isdigit():
            count += char
        else:
            decoded_text += char * int(count)
            count = ''
    return decoded_text

def compress_text_rle():
    input_text = text_input.get("1.0", tk.END).strip()
    if not input_text:
        messagebox.showwarning("Warning", "Please enter some text.")
        return
    
    encoded_text, bits_before, bits_after, compression_ratio, probabilities, entropy, average_length, efficiency= encode_rle(input_text)
    decoded_text = decode_rle(encoded_text)

    output_text = f"Original text: {input_text}\n"
    output_text += f"Encoded text: {encoded_text}\n"
    output_text += f"Decoded text: {decoded_text}\n"
    output_text += f"Bits before encoding: {bits_before}\n"
    output_text += f"Bits after encoding: {bits_after}\n"
    output_text += f"Compression ratio (%): {compression_ratio}\n"
    output_text += "Probability of occurrence for each character:\n"
    for char, prob in probabilities.items():
        output_text += f"- {char}: {prob}\n"
    output_text += f"Entropy: {entropy}\n"
    output_text += f"Average length: {average_length}\n"
    output_text += f"Efficiency of this message: {efficiency}\n"
    
    text_output.delete("1.0", tk.END)
    text_output.insert("1.0", output_text)

    # Clear the canvas
    canvas.delete("all")


    

# Text Arithmetic

In [31]:
efficiency_arith = None 

class ArithmeticCoding:
    def __init__(self, probabilities):
        self.probabilities = probabilities
        self.low_range = {}
        self.high_range = {}
        self.cumulative_prob = {}
        self.initialize_ranges()

    def initialize_ranges(self):
        low = 0.0
        for symbol, prob in self.probabilities.items():
            self.low_range[symbol] = low
            self.high_range[symbol] = low + prob
            self.cumulative_prob[symbol] = low
            low += prob

    def encode(self, sequence):
        low = 0.0
        high = 1.0
        for symbol in sequence:
            range_width = high - low
            high = low + range_width * self.high_range[symbol]
            low = low + range_width * self.low_range[symbol]
        return (low + high) / 2

    def decode(self, code, length):
        sequence = []
        for _ in range(length):
            found_symbol = False
            for symbol, low in self.low_range.items():
                if low <= code < self.high_range[symbol]:
                    sequence.append(symbol)
                    range_width = self.high_range[symbol] - low
                    code = (code - low) / range_width
                    found_symbol = True
                    break
            if not found_symbol:
                raise ValueError("Invalid compression code")
        return ''.join(sequence)
    
    def encode_length(self, symbol):
        # Implement logic to calculate the length of the encoded code for the symbol
        encoded_code = self.encode(symbol)
        # Example: return the length of the string representation of the float
        return len(str(encoded_code))

def get_probabilities():
    probabilities = {}
    prob_input = text_probabilities.get("1.0", tk.END).strip()
    prob_lines = prob_input.split("\n")
    for line in prob_lines:
        symbol, prob = line.split(":")
        probabilities[symbol.strip()] = float(prob)
    return probabilities

def calculate_metrics(original_text, compressed_code, arithmetic_coder):
    original_text_bits = len(original_text) * 8  # Assuming 8 bits per character
    compressed_bits = len(compressed_code)
    compression_ratio = (original_text_bits / compressed_bits) * 100
    probabilities = get_probabilities()
    entropy = calculate_entropy_arith(probabilities)
    
    average_length = calculate_average_length_arith(probabilities, arithmetic_coder)
    efficiency_arith = entropy / average_length
    return original_text_bits, compressed_bits, compression_ratio, entropy, average_length, efficiency_arith

def calculate_entropy_arith(probabilities):
    # Calculate the entropy using the probabilities
    entropy = -sum(prob * math.log2(prob) for prob in probabilities.values())
    return entropy

def calculate_average_length_arith(probabilities, arithmetic_coder):
    # Calculate the average length of the encoded symbols
    average_length = sum(arithmetic_coder.encode_length(symbol) * prob for symbol, prob in probabilities.items())
    return average_length

def compress_text_arithmetic():
    global efficiency_arith
    input_text = text_input.get("1.0", tk.END).strip()
    if not input_text:
        messagebox.showwarning("Warning", "Please enter some text.")
        return

    probabilities = get_probabilities()

    arithmetic_coder = ArithmeticCoding(probabilities)
    compressed_code = str(arithmetic_coder.encode(input_text))  # Convert to string

    original_text_bits, compressed_bits, compression_ratio, entropy, average_length, efficiency_arith = calculate_metrics(input_text, compressed_code, arithmetic_coder)

    output_text = f"Original text: {input_text}\n"
    output_text += f"Compressed code: {compressed_code}\n"
    output_text += f"Original text bits: {original_text_bits}\n"
    output_text += f"Compressed bits: {compressed_bits}\n"
    output_text += f"Compression Ratio: {compression_ratio:.2f}%\n"
    output_text += f"Entropy: {entropy:.2f}\n"
    output_text += f"Average Length: {average_length:.2f}\n"
    output_text += f"Efficiency: {efficiency_arith:.2f}\n"

    text_output.delete("1.0", tk.END)
    text_output.insert("1.0", output_text)

# Golomb-Rice

In [32]:
efficiency_golomb = None  # Define efficiency as a global variable

# Golomb-Rice encoding functions
def golomb_rice_encode(sequence, M):
    # Generate RLE from the sequence
    rle_sequence_golomb = []
    char_sequence = []
    count = 1
    for i in range(1, len(sequence)):
        if sequence[i] == sequence[i - 1]:
            count += 1
        else:
            rle_sequence_golomb.append((sequence[i-1], count))  # Storing both symbol and count
            count = 1
    rle_sequence_golomb.append((sequence[-1], count))  # Storing the last symbol and its count

    if not rle_sequence_golomb:
        raise ValueError("Empty sequence.")

    codewords = {}
    for char, count in rle_sequence_golomb:
        # Generate Golomb-Rice codeword for count
        count_codeword = golomb_rice_encode_count(count, M)
        # Store the Golomb-Rice codeword along with the character
        codewords[(char, count)] = count_codeword

    # Return the encoded sequence and the codewords
    return rle_sequence_golomb, codewords

def golomb_rice_decode(encoded, codewords, M):
    decoded_sequence = []
    for char, count in encoded:
        # Decode the count using the Golomb-Rice codeword
        count_codeword = codewords[(char, count)]
        decoded_value = golomb_rice_decode_count(count_codeword, M)
        # Append the decoded value to the sequence
        decoded_sequence.extend([char] * decoded_value)
    # Return the decoded sequence
    return decoded_sequence

def golomb_rice_encode_count(value, M):
    quotient = value // M
    remainder = value % M
    
    # Encode quotient in unary; a sequence of 1s followed by a 0.
    unary = '1' * quotient + '0'
    
    # Encode remainder in binary. The length of the binary representation is log2(M).
    binary_length = math.ceil(math.log2(M))
    binary = format(remainder, f'0{binary_length}b')
    
    return unary + binary

def golomb_rice_decode_count(encoded, M):
    decoded_value = 0
    # Extract the quotient
    while encoded[0] == '1':
        decoded_value += 1
        encoded = encoded[1:]
    # Skip over the separator '0'
    encoded = encoded[1:]
    # Extract the binary part
    remainder = ''
    while len(remainder) < math.ceil(math.log2(M)):
        remainder += encoded[0]
        encoded = encoded[1:]
    # Convert the binary part to integer
    remainder_value = int(remainder, 2)
    # Decode the value using the quotient and remainder
    decoded_value = decoded_value * M + remainder_value
    return decoded_value


def calculate_average_length_golomb(rle_sequence_golomb, codewords, probabilities):
    total_length = 0

    for char, count in rle_sequence_golomb:
        probability = probabilities[char]
        codeword_length = len(codewords[(char, count)])
        total_length += probability * codeword_length

    return total_length


def process_golomb_text(operation):
    global efficiency_golomb 
    canvas.delete("all")
    try:
        sequence = text_input.get("1.0", tk.END).strip()
        M = int(entry_M.get())
        if M <= 0:
            raise ValueError("M should be a positive integer.")

        # Generate RLE from the sequence
        rle_sequence_golomb, codewords = golomb_rice_encode(sequence, M)

        if not rle_sequence_golomb:
            raise ValueError("Empty sequence.")

       # Calculate bits before encoding
        if all(char in '01' for char in sequence):
            original_bits = len(sequence)  # Each character is considered as one bit
        else:
            original_bits = len(sequence) * 8  # Each character is considered as one byte (eight bits)

        # Calculate bits after encoding
        encoded_bits = sum(len(codewords[(char, count)]) for char, count in rle_sequence_golomb)
        encoded_code = ''.join(codewords[(char, count)] for char, count in rle_sequence_golomb)

        if operation == "compress":
            decoded_text = golomb_rice_decode(rle_sequence_golomb, codewords, M)
            decoded_sequence = ''.join(decoded_text)
            additional_info = f"Decoded text: {decoded_sequence}\n"
        else:
            additional_info = ""
            

        # Calculate compression ratio
        compression_ratio = original_bits/encoded_bits
               
        # Calculate probabilities
        probabilities = calculate_probability(sequence)

        # Calculate entropy
        # Calculate entropy of the original text
        entropy = calculate_entropy(probabilities)

        # Calculate average length
        average_length = calculate_average_length_golomb(rle_sequence_golomb, codewords, probabilities)

        # Calculate efficiency
        efficiency_golomb = entropy / average_length * 100

        # Prepare output text
        output_text = f"Original sequence: {sequence}\n"
        output_text += f"Encoded code:\n{encoded_code}\n"
        output_text += additional_info
        output_text += f"Character Probabilities:\n"
        # Calculate probabilities of characters in the original text
        for char, prob in probabilities.items():
            output_text += f"- {char}: {prob}\n"
        output_text += f"Codewords:\n"
        for (char, count), codeword in codewords.items():
            output_text += f"- {char} ({count}): {codeword}\n"
        output_text += f"Bits before encoding: {original_bits}\n"
        output_text += f"Bits after encoding: {encoded_bits}\n"
        output_text += f"Compression ratio: {compression_ratio:.2f}\n"
        output_text += f"Entropy: {entropy:.2f} bits/character\n"
        output_text += f"Average Length: {average_length:.2f} bits/character\n"
        output_text += f"Efficiency: {efficiency_golomb:.2f}%\n"

        text_output.delete("1.0", tk.END)
        text_output.insert("1.0", output_text)
        
    except ValueError as e:
        messagebox.showerror("Error", str(e))


# LZW 

In [37]:
efficiency_lzw = None 

def LZW_compress(text):
    # Initialize dictionary with ASCII characters
    dictionary_size = 256
    dictionary = {chr(i): i for i in range(dictionary_size)}
    result = []
    current_string = ""
    for char in text:
        new_string = current_string + char
        if new_string in dictionary:
            current_string = new_string
        else:
            result.append(dictionary[current_string])
            # Add new string to dictionary
            dictionary[new_string] = dictionary_size
            dictionary_size += 1
            current_string = char
    if current_string:
        result.append(dictionary[current_string])
    return result

def calculate_entropy(probabilities):
    entropy = 0
    for prob in probabilities.values():
        entropy -= prob * math.log2(prob)
    return entropy

def calculate_efficiency(average_length, entropy):
    return entropy / average_length

def compress_text_lzw():
    global efficiency_lzw
    input_text = text_input.get("1.0", tk.END).strip()
    if not input_text:
        messagebox.showwarning("Warning", "Please enter some text.")
        return

    compressed_text = LZW_compress(input_text)

    # Calculate probabilities of characters in the original text
    probabilities = calculate_probability(input_text)

    # Calculate entropy of the original text
    entropy = calculate_entropy(probabilities)

    # Calculate average length of the compressed LZW output
    average_length = len(compressed_text) *8  # Assuming each code takes 12 bits

    # Calculate efficiency of the compressed LZW output
    efficiency_lzw = calculate_efficiency(average_length, entropy)

    bits_before = len(input_text) * 8
    bits_after = average_length
    compression_ratio = (bits_before / bits_after)*100

    output_text = f"Original text: {input_text}\n"
    output_text += f"Compressed text: {compressed_text}\n"
    output_text += f"Bits before encoding: {bits_before}\n"
    output_text += f"Bits after encoding: {bits_after}\n"
    output_text += f"Compression ratio (%): {compression_ratio}%\n"
    output_text += "Probability of occurrence for each character:\n"
    for char, prob in probabilities.items():
        output_text += f"- {char}: {prob}\n"
    output_text += f"Entropy: {entropy}\n"
    output_text += f"Average length: {average_length}\n"
    output_text += f"Efficiency of this message: {efficiency_lzw}\n"

    text_output.delete("1.0", tk.END)
    text_output.insert("1.0", output_text)


# Optimal Technique

In [34]:
def choose_optimal_technique():
    input_text = text_input.get("1.0", tk.END).strip()
    if not input_text:
        messagebox.showwarning("Warning", "Please enter some text.")
        return

    # Calculate probabilities and other metrics for Huffman encoding
    probabilities_huffman = calculate_probability(input_text)
    huffman_tree = build_huffman_tree(probabilities_huffman)
    huffman_codes = {}
    generate_huffman_codes(huffman_tree, "", huffman_codes)
    average_length_huffman = calculate_average_length(probabilities_huffman, huffman_codes)
    entropy_huffman = calculate_entropy(probabilities_huffman)
    efficiency_huffman = calculate_efficiency(average_length_huffman, entropy_huffman)

    # Calculate probabilities and other metrics for RLE encoding
    _, _, _, _, probabilities_rle, entropy_rle, average_length_rle, efficiency_rle = encode_rle(input_text)
    
    # Choose the optimal technique based on efficiency
    optimal_technique = max(
        ("Golomb-Rice Encoding", efficiency_golomb),
        ("Huffman Encoding", efficiency_huffman),
        ("Run Length Encoding (RLE)", efficiency_rle),
        ("Arithmetic Encoding", efficiency_arith),
        ("LZW Compression", efficiency_lzw),
        key=lambda x: x[1]
    )

    messagebox.showinfo("Optimal Technique", f"{optimal_technique[0]} is recommended for this text.")

# GUI

In [38]:
root = tk.Tk()
root.title("Compression Tool")

frame_input = tk.Frame(root)
frame_input.pack(padx=10, pady=10)

label_input = tk.Label(frame_input, text="Enter the text to be compressed:")
label_input.pack()

text_input = tk.Text(frame_input, height=5, width=50)
text_input.pack()

label_probabilities = tk.Label(frame_input, text="Enter probabilities (e.g., 'a:0.5\nb:0.3\nc:0.2'):")
label_probabilities.pack()

text_probabilities = tk.Text(frame_input, height=5, width=50)
text_probabilities.pack()

label_M = tk.Label(frame_input, text="Enter the M parameter:")
label_M.pack()
entry_M = tk.Entry(frame_input)
entry_M.pack()

def compress_decode_huffman():
    button_text = button_compress_decode_huffman.cget("text")
    if button_text == "Compress (Huffman)":
        process_huffman_text("compress")
        button_compress_decode_huffman.config(text="Decode (Huffman)")
    else:
        process_huffman_text("decode")
        button_compress_decode_huffman.config(text="Compress (Huffman)")

button_compress_decode_huffman = tk.Button(root, text="Compress (Huffman)", command=compress_decode_huffman)
button_compress_decode_huffman.pack(pady=5)

button_compress_rle = tk.Button(root, text="Compress (Run Length Encoding)", command=compress_text_rle)
button_compress_rle.pack(pady=5)

button_compress_arithmetic = tk.Button(root, text="Compress (Arithmetic Encoding)", command=compress_text_arithmetic)
button_compress_arithmetic.pack(pady=5)

def compress_decode_golomb():
    button_text = button_compress_decode_golomb.cget("text")
    if button_text == "Compress (Golomb)":
        process_golomb_text("compress")
        button_compress_decode_golomb.config(text="Decode (Golomb)")
    else:
        process_golomb_text("decode")
        button_compress_decode_golomb.config(text="Compress (Golomb)")
        
button_compress_decode_golomb = tk.Button(root, text="Compress (Golomb)", command=compress_decode_golomb)
button_compress_decode_golomb.pack(pady=5)

button_compress_lzw = tk.Button(root, text="Compress (LZW)", command=compress_text_lzw)
button_compress_lzw.pack(pady=5)

button_optimal_technique = tk.Button(root, text="Choose Optimal Technique", command=choose_optimal_technique)
button_optimal_technique.pack(pady=5)

frame_output = tk.Frame(root)
frame_output.pack(padx=10, pady=10)

label_output = tk.Label(frame_output, text="Compression/Decompression Details:")
label_output.pack()

text_output = tk.Text(frame_output, height=20, width=90)
text_output.pack()

canvas = tk.Canvas(root, width=800, height=600)
canvas.pack()

root.mainloop()